# Machine Learning Lab Project - Credit Card Overdue Likelihood Prediction
##  Business Understanding

## Data Understanding
The data for this task is taken from [this](https://www.kaggle.com/datasets/rikdifos/credit-card-approval-prediction) kaggle dataset. The kaggle page provides two `.csv` files:
- application_record.csv
- credit_record.csv

On a simple level, `application_record.csv` contains the customer data and `credit_record.csv` contains the customers credit history. The specific content is now investigated further.

For beeing able to analyse the datasets, the necessary libraries are imported first:

In [254]:
import pandas as pd

In this next step the two `.csv` files are loaded into a pandas datafram. This enables an analysis with the full pandas funcionality, which makes the data understanding process way easier.

In [255]:
customer_df = pd.read_csv("Data/application_record.csv")
credit_df = pd.read_csv("Data/credit_record.csv")

### application_record.csv
First, it is important to analyse the columns of the dataset:

In [256]:
customer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438557 entries, 0 to 438556
Data columns (total 18 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   ID                   438557 non-null  int64  
 1   CODE_GENDER          438557 non-null  object 
 2   FLAG_OWN_CAR         438557 non-null  object 
 3   FLAG_OWN_REALTY      438557 non-null  object 
 4   CNT_CHILDREN         438557 non-null  int64  
 5   AMT_INCOME_TOTAL     438557 non-null  float64
 6   NAME_INCOME_TYPE     438557 non-null  object 
 7   NAME_EDUCATION_TYPE  438557 non-null  object 
 8   NAME_FAMILY_STATUS   438557 non-null  object 
 9   NAME_HOUSING_TYPE    438557 non-null  object 
 10  DAYS_BIRTH           438557 non-null  int64  
 11  DAYS_EMPLOYED        438557 non-null  int64  
 12  FLAG_MOBIL           438557 non-null  int64  
 13  FLAG_WORK_PHONE      438557 non-null  int64  
 14  FLAG_PHONE           438557 non-null  int64  
 15  FLAG_EMAIL       

As can be seen above, the dataset consists of 17 columns, containing numeral as well as textual data. It also seems as if there is already an unique identifier for every customer in the column `ID`. A concrete description fo these different columns can be retrieved from the datasets [kaggle page](https://www.kaggle.com/datasets/rikdifos/credit-card-approval-prediction/data):

|Feature name|Explanation|Remarks|
|:-----------|:----------|:------|
|ID 	     |Client number||
|CODE_GENDER |	Gender 	||
|FLAG_OWN_CAR| 	Is there a car 	||
|FLAG_OWN_REALTY| 	Is there a property|| 	
|CNT_CHILDREN| 	Number of children 	||
|AMT_INCOME_TOTAL| 	Annual income 	||
|NAME_INCOME_TYPE| 	Income category 	||
|NAME_EDUCATION_TYPE| 	Education level ||	
|NAME_FAMILY_STATUS| 	Marital status 	||
|NAME_HOUSING_TYPE| 	Way of living 	||
|DAYS_BIRTH| 	Birthday |	Count backwards from current day (0), -1 means yesterday|
|DAYS_EMPLOYED| 	Start date of employment |	Count backwards from current day(0). If positive, it means the person currently  unemployed.|
|FLAG_MOBIL| 	Is there a mobile phone 	||
|FLAG_WORK_PHONE| 	Is there a work phone 	||
|FLAG_PHONE| 	Is there a phone 	||
|FLAG_EMAIL| 	Is there an email 	||
|OCCUPATION_TYPE| 	Occupation 	||
|CNT_FAM_MEMBERS| 	Family size||

Now that the purpose of the columns is clear, the actual data can be analyzed:

In [257]:
customer_df.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0


From these first five entries several things can be observed:
1. The ID does not start from 0 but seems to be unique.
2. For the `CODE_GENDER` the flags `F` (Female) and `M` (Male) are used.
3. For `FLAG_OWN_CAR` and `FLAG_OWN_REALTY` the flags `Y` (Yes) and `N` (No) are used.
4. For `NAME_INCOME_TYPE`, `NAME_EDUCATION_TYPE`, `NAME_FAMILY_STATUS` and `NAME_HOUSING_TYPE` are textual fields, but seem to have only a few different values.
5. `OCCUPATION_TYPE` is a textual field and the values seem to be very different ("Freetext Field").
6. For `FLAG_MOBIL`, `FLAG_WORK_PHONE`, `FLAG_PHONE` and `FLAG_EMAIL` the flags 1 (Yes) and 0 (No) are used.

Before basing the data perparation on these findings, the assumptions have to be validated:

#### 1. Unique `ID`

In [258]:
customer_df['ID'].value_counts()

ID
7137299    2
7702238    2
7282535    2
7243768    2
7050948    2
          ..
5690727    1
6621262    1
6621261    1
6621260    1
6842885    1
Name: count, Length: 438510, dtype: int64

The ouptut of the value count clearly shows that not every ID is unique. As some IDs are contained twice, the real amout of customers contained in the dataset is only 438510.
#### 2. Gender

In [259]:
customer_df['CODE_GENDER'].value_counts()

CODE_GENDER
F    294440
M    144117
Name: count, dtype: int64

For the gender the assumption that only the flags `F` and `M` are used was correct.
#### 3. Flag Car / Real-Estate

In [260]:
print(customer_df['FLAG_OWN_CAR'].value_counts())
print(customer_df['FLAG_OWN_REALTY'].value_counts())

FLAG_OWN_CAR
N    275459
Y    163098
Name: count, dtype: int64
FLAG_OWN_REALTY
Y    304074
N    134483
Name: count, dtype: int64


Also for the flags for the possesion of car and real-estate the assumption that there are only the flags `Y` and `N` was correct.
#### 4. Text fields income, education, family and housing

In [261]:
customer_df['NAME_INCOME_TYPE'].value_counts()

NAME_INCOME_TYPE
Working                 226104
Commercial associate    100757
Pensioner                75493
State servant            36186
Student                     17
Name: count, dtype: int64

As can be seen above, there are five different types of income. This means the column can be encoded without any problems and can be used for the modeling later.

In [262]:
customer_df['NAME_EDUCATION_TYPE'].value_counts()

NAME_EDUCATION_TYPE
Secondary / secondary special    301821
Higher education                 117522
Incomplete higher                 14851
Lower secondary                    4051
Academic degree                     312
Name: count, dtype: int64

The education field also consists of only five types and can therefore also be used for modelling without any problems.

In [263]:
customer_df['NAME_FAMILY_STATUS'].value_counts()

NAME_FAMILY_STATUS
Married                 299828
Single / not married     55271
Civil marriage           36532
Separated                27251
Widow                    19675
Name: count, dtype: int64

Also the family status has five different values.

In [264]:
customer_df['NAME_HOUSING_TYPE'].value_counts()

NAME_HOUSING_TYPE
House / apartment      393831
With parents            19077
Municipal apartment     14214
Rented apartment         5974
Office apartment         3922
Co-op apartment          1539
Name: count, dtype: int64

The housing has six different values, which is still acceptable.
#### 5. Text field occupation type

In [265]:
print(customer_df['OCCUPATION_TYPE'].value_counts())
print("Amount of null values: ", customer_df['OCCUPATION_TYPE'].isnull().sum())

OCCUPATION_TYPE
Laborers                 78240
Core staff               43007
Sales staff              41098
Managers                 35487
Drivers                  26090
High skill tech staff    17289
Accountants              15985
Medicine staff           13520
Cooking staff             8076
Security staff            7993
Cleaning staff            5845
Private service staff     3456
Low-skill Laborers        2140
Secretaries               2044
Waiters/barmen staff      1665
Realty agents             1041
HR staff                   774
IT staff                   604
Name: count, dtype: int64
Amount of null values:  134203


Even though there are way less different values than expected (only 18), the column contains many null values, which may make it difficult to work with it.
#### 6. Contact method flags

In [266]:
print(customer_df['FLAG_MOBIL'].value_counts())
print(customer_df['FLAG_WORK_PHONE'].value_counts())
print(customer_df['FLAG_PHONE'].value_counts())
print(customer_df['FLAG_EMAIL'].value_counts())

FLAG_MOBIL
1    438557
Name: count, dtype: int64
FLAG_WORK_PHONE
0    348156
1     90401
Name: count, dtype: int64
FLAG_PHONE
0    312353
1    126204
Name: count, dtype: int64
FLAG_EMAIL
0    391102
1     47455
Name: count, dtype: int64


As expected, these columns only use the flags `1` and `0`. On top of that, column `FLAG_MOBIL` only contains the value `1`, which means all customers at least are registered with a mobile phone. Therefore, this column can be left out completely.
### credit_record.csv
Again, first analyze the columns:

In [267]:
credit_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 3 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   ID              1048575 non-null  int64 
 1   MONTHS_BALANCE  1048575 non-null  int64 
 2   STATUS          1048575 non-null  object
dtypes: int64(2), object(1)
memory usage: 24.0+ MB


As can be seen above, this table only contains three columns. According to the [kaggle page](https://www.kaggle.com/datasets/rikdifos/credit-card-approval-prediction/data), these columns mean the following:

|Feature name| 	Explanation |	Remarks|
|:-|:-|:-|
|ID |	Client number 	||
|MONTHS_BALANCE |	Record month |	The month of the extracted data is the starting point, backwards, 0 is the current month, -1 is the previous month, and so on|
|STATUS |	Status| 	0: 1-29 days past due 1: 30-59 days past due 2: 60-89 days overdue 3: 90-119 days overdue 4: 120-149 days overdue 5: Overdue or bad debts, write-offs for more than 150 days C: paid off that month X: No loan for the month|

Checking for unique `ID` values now reveals for how many customers there exists credit data:

In [268]:
len(credit_df['ID'].unique())

45985

There is only credit data for 45985 customers, that means only parts of the `customer_df` can be used.
As a last step check the values of the `STATUS` column:

In [269]:
credit_df['STATUS'].value_counts()

STATUS
C    442031
0    383120
X    209230
1     11090
5      1693
2       868
3       320
4       223
Name: count, dtype: int64

The information given on the kaggle page is correct, only the stated flags are used.
### Summary
Overall, the dataset consists of two parts: the customer data and the credit data. The customer data mostly contains information about income, job, family situation and contact methods as these are important aspects for evaluating the creditworthiness. The credit data is basically a credit history overview, showing for a given custumer and month if the credit was paid back on time. This credit data can now be used for calculating an "overdue_likelyhood" for every customer which states how likely it is for this specific customer to not pay it's credit back in time. This is an important information for a credit institute. Based on all the findings in this section the two datasets can now be prepared, connected and finally used for training a machine learning model.

## Data Preparation

Get the overdue likelyhood for the customers:

In [270]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer

grouped_df = credit_df.groupby('ID')['STATUS'].value_counts().unstack(fill_value=0)

grouped_df['overdue_likelihood'] = 1 - (grouped_df['C'] / (grouped_df.sum(axis=1) - grouped_df['X']))

result_df = grouped_df.reset_index()[['ID', 'overdue_likelihood']]

print(result_df.head())

nan_count = result_df['overdue_likelihood'].isna().sum()

print(f'The number of entries with NaN values in overdue_likelihood: {nan_count}')


STATUS       ID  overdue_likelihood
0       5001711            1.000000
1       5001712            0.526316
2       5001713                 NaN
3       5001714                 NaN
4       5001715                 NaN
The number of entries with NaN values in overdue_likelihood: 4536


That means ca. 33110 customers are really usable.

Remove NaN entrys

In [271]:
result_df.dropna(subset=['overdue_likelihood'], inplace=True)
nan_count = result_df['overdue_likelihood'].isna().sum()

print(f'The number of entries with NaN values in overdue_likelihood: {nan_count}')

The number of entries with NaN values in overdue_likelihood: 0


Merge the data:

In [272]:
merged_df = pd.merge(customer_df, result_df, on='ID', how='inner')
print(merged_df.head())
#print(len(merged_df['ID'].unique()))

        ID CODE_GENDER FLAG_OWN_CAR FLAG_OWN_REALTY  CNT_CHILDREN  \
0  5008804           M            Y               Y             0   
1  5008805           M            Y               Y             0   
2  5008806           M            Y               Y             0   
3  5008808           F            N               Y             0   
4  5008810           F            N               Y             0   

   AMT_INCOME_TOTAL      NAME_INCOME_TYPE            NAME_EDUCATION_TYPE  \
0          427500.0               Working               Higher education   
1          427500.0               Working               Higher education   
2          112500.0               Working  Secondary / secondary special   
3          270000.0  Commercial associate  Secondary / secondary special   
4          270000.0  Commercial associate  Secondary / secondary special   

     NAME_FAMILY_STATUS  NAME_HOUSING_TYPE  DAYS_BIRTH  DAYS_EMPLOYED  \
0        Civil marriage   Rented apartment      -12005 

In [273]:
bins = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
labels = ['0-10%', '10-20%', '20-30%', '30-40%', '40-50%', '50-60%', '60-70%', '70-80%', '80-90%', '90-100%']

#bins = [0, 0.2, 0.4, 0.6,  0.8, 1]
#labels = ['1', '2', '3', '4', '5']

merged_df['overdue_class'] = pd.cut(merged_df['overdue_likelihood'], bins=bins, labels=labels, include_lowest=True)
merged_df['overdue_class'].value_counts()

overdue_class
90-100%    15462
10-20%      3423
20-30%      2916
0-10%       2715
30-40%      2546
40-50%      1984
50-60%      1300
60-70%      1111
70-80%      1012
80-90%       641
Name: count, dtype: int64

In [274]:
one_hot_cols = ['NAME_INCOME_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE']
ordinal_cols = ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_EDUCATION_TYPE']


merged_df = merged_df.drop('OCCUPATION_TYPE', axis=1)
merged_df = merged_df.drop('overdue_likelihood', axis=1)
merged_df = merged_df.drop('ID', axis=1)

df_ord = merged_df.copy()
ordinal_encoder = OrdinalEncoder()
df_ord[ordinal_cols] = ordinal_encoder.fit_transform(df_ord[ordinal_cols])

df_enc = pd.get_dummies(df_ord, columns=one_hot_cols)

print(df_enc.head())

   CODE_GENDER  FLAG_OWN_CAR  FLAG_OWN_REALTY  CNT_CHILDREN  AMT_INCOME_TOTAL  \
0          1.0           1.0              1.0             0          427500.0   
1          1.0           1.0              1.0             0          427500.0   
2          1.0           1.0              1.0             0          112500.0   
3          0.0           0.0              1.0             0          270000.0   
4          0.0           0.0              1.0             0          270000.0   

   NAME_EDUCATION_TYPE  DAYS_BIRTH  DAYS_EMPLOYED  FLAG_MOBIL  \
0                  1.0      -12005          -4542           1   
1                  1.0      -12005          -4542           1   
2                  4.0      -21474          -1134           1   
3                  4.0      -19110          -3051           1   
4                  4.0      -19110          -3051           1   

   FLAG_WORK_PHONE  ...  NAME_FAMILY_STATUS_Married  \
0                1  ...                       False   
1           

In [275]:
X_enc = df_enc.drop('overdue_class', axis=1)
Y = df_enc['overdue_class']

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_enc)
df_scaled = pd.DataFrame(X_scaled, columns= X_enc.columns)
print(df_scaled.head())

   CODE_GENDER  FLAG_OWN_CAR  FLAG_OWN_REALTY  CNT_CHILDREN  AMT_INCOME_TOTAL  \
0          1.0           1.0              1.0           0.0          0.258721   
1          1.0           1.0              1.0           0.0          0.258721   
2          1.0           1.0              1.0           0.0          0.055233   
3          0.0           0.0              1.0           0.0          0.156977   
4          0.0           0.0              1.0           0.0          0.156977   

   NAME_EDUCATION_TYPE  DAYS_BIRTH  DAYS_EMPLOYED  FLAG_MOBIL  \
0                 0.25    0.753539       0.029324         0.0   
1                 0.25    0.753539       0.029324         0.0   
2                 1.00    0.210810       0.038270         0.0   
3                 1.00    0.346306       0.033237         0.0   
4                 1.00    0.346306       0.033237         0.0   

   FLAG_WORK_PHONE  ...  NAME_FAMILY_STATUS_Married  \
0              1.0  ...                         0.0   
1           

# TODO:
- overdue-likelyhood zu labels
- encoden
    - Eig geht fast alles ordinal encoded
    - OCCUPATION_TYPE fliegt ganz raus
- scalen
- random forest




## Modeling

In [276]:
from sklearn.model_selection import train_test_split, GridSearchCV
X_train, X_test, y_train, y_test = train_test_split(df_scaled, Y, test_size=0.3, random_state=42)

In [277]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np

#rf_classifier = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=0, class_weight='balanced')

#rf_classifier.fit(X_train, y_train)
#y_test_pred = rf_classifier.predict(X_test)
#accuracy = accuracy_score(y_test, y_test_pred)


param_grid = {'n_estimators': np.array([25, 50, 100]), 
              'criterion':['gini','entropy'],
              'min_samples_leaf':[3,5,10],
              'max_depth': [1,2,3,4],
              'class_weight':['balanced']}

forest = RandomForestClassifier(max_features='sqrt',
                                random_state=1,                                
                                n_jobs=-1)

forest_cv = GridSearchCV(forest, param_grid, scoring='accuracy', cv=10, verbose=3, n_jobs=-1, refit=True) #refit important.
forest_cv.fit(X_train, y_train)

print (forest_cv.best_estimator_)

y_pred = forest_cv.best_estimator_.predict(X_test)
accuracy_score(y_test,y_pred)



#print(accuracy)
#print(y_test_pred)
#print(y_test)

Fitting 10 folds for each of 240 candidates, totalling 2400 fits


KeyboardInterrupt: 

## Evaluation